### Kinematics
```
Kinematics in Robotics defines a mapping between configuration space and task space
```
- `Configuration space`: space of (controllable) robot joint angles
- `Task space`: Cartesian space (e.g., end effector poses)
- Forward kinematics (FK) maps robot joint angles to the end-effector poses (i.e., positions and orientations)
- Inverse kinematics (IK) maps the end-effector poses to the robot joint angles (solution may not exist)
- Kinematics is not affected by Gravity

In our implementation, kinematic simulation is done by `env.forward` and internally `mujoco.mj_forward` is called

In [33]:
import sys,mujoco
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
from mujoco_parser import *
from slider import * # <= slider control
from utility import *
print ("MuJoCo:[%s]"%(mujoco.__version__))

MuJoCo:[3.1.1]


#### Parse `Unitree G1`

In [34]:
xml_path = '../asset/snapbot/scene_snapbot.xml'
env = MuJoCoParserClass(name='Snapbot',rel_xml_path=xml_path,verbose=True)

name:[Snapbot] dt:[0.002] HZ:[500]
n_qpos:[25] n_qvel:[24] n_qacc:[24] n_ctrl:[8]

n_body:[24]
 [0/24] [world] mass:[0.00]kg
 [1/24] [torso] mass:[0.24]kg
 [2/24] [Camera_module_1] mass:[0.06]kg
 [3/24] [Camera_module_2] mass:[0.06]kg
 [4/24] [Leg_module_1_1] mass:[0.06]kg
 [5/24] [Leg_module_1_2] mass:[0.08]kg
 [6/24] [Leg_module_1_3] mass:[0.02]kg
 [7/24] [Leg_module_1_4] mass:[0.01]kg
 [8/24] [Leg_module_1_4bar] mass:[0.01]kg
 [9/24] [Leg_module_2_1] mass:[0.06]kg
 [10/24] [Leg_module_2_2] mass:[0.08]kg
 [11/24] [Leg_module_2_3] mass:[0.02]kg
 [12/24] [Leg_module_2_4] mass:[0.01]kg
 [13/24] [Leg_module_2_4bar] mass:[0.01]kg
 [14/24] [Leg_module_4_1] mass:[0.06]kg
 [15/24] [Leg_module_4_2] mass:[0.08]kg
 [16/24] [Leg_module_4_3] mass:[0.02]kg
 [17/24] [Leg_module_4_4] mass:[0.01]kg
 [18/24] [Leg_module_4_4bar] mass:[0.01]kg
 [19/24] [Leg_module_5_1] mass:[0.06]kg
 [20/24] [Leg_module_5_2] mass:[0.08]kg
 [21/24] [Leg_module_5_3] mass:[0.02]kg
 [22/24] [Leg_module_5_4] mass:[0.01]kg
 [

#### Slider control of revolute joints of `G1`

In [35]:
# Initialize slider control
env.reset(step=True)
init_qpos = env.get_qpos_joints(joint_names=env.rev_joint_names)
sliders = MultiSliderClass(
    n_slider      = env.n_rev_joint,
    title         = 'Sliders for [%s] Control'%(env.name),
    window_width  = 600,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 350,
    label_texts   = env.rev_joint_names,
    slider_mins   = env.rev_joint_mins,
    slider_maxs   = env.rev_joint_maxs,
    slider_vals   = init_qpos,
    verbose       = False,
)
idxs_fwd = env.get_idxs_fwd(joint_names=env.rev_joint_names)
# Loop
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    # Update
    sliders.update()
    env.forward(q=sliders.get_slider_values(),joint_idxs=idxs_fwd)
    # Render
    if env.loop_every(tick_every=10):
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.plot_contact_info()
        env.render()
# Close slider
sliders.close()
print ("Done.")

Done.


#### Parse `UR5e`

In [30]:
xml_path = '../asset/tabletop_manipulation/scene_ur5e_objects.xml'
env = MuJoCoParserClass(name='UR5e',rel_xml_path=xml_path,verbose=True)

name:[UR5e] dt:[0.002] HZ:[500]
n_qpos:[33] n_qvel:[30] n_qacc:[30] n_ctrl:[7]

n_body:[27]
 [0/27] [world] mass:[0.00]kg
 [1/27] [front_object_table] mass:[1.00]kg
 [2/27] [side_object_table] mass:[1.00]kg
 [3/27] [ur_base] mass:[4.00]kg
 [4/27] [ur_shoulder_link] mass:[3.70]kg
 [5/27] [ur_upper_arm_link] mass:[8.39]kg
 [6/27] [ur_forearm_link] mass:[2.27]kg
 [7/27] [ur_wrist_1_link] mass:[1.22]kg
 [8/27] [ur_wrist_2_link] mass:[1.22]kg
 [9/27] [ur_wrist_3_link] mass:[0.19]kg
 [10/27] [ur_tcp_link] mass:[0.00]kg
 [11/27] [ur_camera_mount] mass:[0.09]kg
 [12/27] [ur_d435i] mass:[0.07]kg
 [13/27] [ur_rg2_gripper_base_link] mass:[0.20]kg
 [14/27] [ur_camera_center] mass:[0.00]kg
 [15/27] [ur_rg2_gripper_finger1_finger_link] mass:[0.01]kg
 [16/27] [ur_rg2_gripper_finger1_inner_knuckle_link] mass:[0.01]kg
 [17/27] [ur_rg2_gripper_finger1_finger_tip_link] mass:[0.01]kg
 [18/27] [ur_rg2_gripper_finger2_finger_link] mass:[0.01]kg
 [19/27] [ur_rg2_gripper_finger2_inner_knuckle_link] mass:[0.01

#### Slider control of `UR5e` using the same code used for `G1`

In [31]:
# Initialize slider control
env.reset(step=True)
init_qpos = env.get_qpos_joints(joint_names=env.rev_joint_names)
sliders = MultiSliderClass(
    n_slider      = env.n_rev_joint,
    title         = 'Sliders for [%s] Control'%(env.name),
    window_width  = 600,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 300,
    label_texts   = env.rev_joint_names,
    slider_mins   = env.rev_joint_mins,
    slider_maxs   = env.rev_joint_maxs,
    slider_vals   = init_qpos,
    verbose       = False,
)
idxs_fwd = env.get_idxs_fwd(joint_names=env.rev_joint_names)
# Loop
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    # Update
    sliders.update()
    env.forward(q=sliders.get_slider_values(),joint_idxs=idxs_fwd)
    # Render
    if env.loop_every(tick_every=10):
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.plot_contact_info()
        env.render()
# Close slider
sliders.close()
print ("Done.")

Done.


#### Arrange objects and render

In [32]:
# Initialize slider control
env.reset(step=True)
init_qpos = env.get_qpos_joints(joint_names=env.rev_joint_names)
sliders = MultiSliderClass(
    n_slider      = env.n_rev_joint,
    title         = 'Sliders for [%s] Control'%(env.name),
    window_width  = 600,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 300,
    label_texts   = env.rev_joint_names,
    slider_mins   = env.rev_joint_mins,
    slider_maxs   = env.rev_joint_maxs,
    slider_vals   = init_qpos,
    verbose       = False,
)
idxs_fwd = env.get_idxs_fwd(joint_names=env.rev_joint_names)

# Arrange objects
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
obj_xyzs = sample_xyzs(
    n_sample  = n_obj,
    x_range   = [+0.6,+1.0],
    y_range   = [-0.45,+0.45],
    z_range   = [0.8,0.81],
    min_dist  = 0.2,
    xy_margin = 0.0
)
for obj_idx in range(n_obj):
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
env.set_geom_color(body_names_to_color=obj_names,rgba_list=get_colors(n_obj))

# Loop
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    # Update
    sliders.update()
    env.forward(q=sliders.get_slider_values(),joint_idxs=idxs_fwd)
    # Render
    if env.loop_every(tick_every=10):
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.plot_contact_info()
        env.render()
# Close slider
sliders.close()
print ("Done.")

Done.
